# Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
%run ../utils/__init__.py
config_logging(logging.INFO)

# Fn to load model

In [ ]:
%run ../models/checkpoint/__init__.py
%run ../utils/files.py

In [ ]:
def _load_model_wrapper(run_name):
    run_id = RunId(run_name, False, 'rg')
    compiled_model = load_compiled_model_report_generation(run_id)
    
    return run_id, compiled_model

# Load data

In [ ]:
%run ../datasets/__init__.py

In [ ]:
dataset_kwargs = {
    'dataset_name': 'iu-x-ray',
    'dataset_type': 'train',
    'max_samples': None,
    'hierarchical': True,
    'frontal_only': True,
    'image_size': (256, 256),
    'norm_by_sample': True,
    'batch_size': 20,
}
dataloader = prepare_data_report_generation(**dataset_kwargs)
dataset = dataloader.dataset
len(dataset)

# Inspect word embeddings

Write embeddings to TB

## Utils

In [ ]:
def plot_emb_distribution(emb, title='Embeddings distribution'):
    if emb.ndim > 1:
        emb = emb.flatten()
    plt.hist(emb, bins=50)
    plt.title(title, fontsize=20)
    plt.ylabel('Count')
    plt.xlabel('Embedding value')

## Select a subsample of words

In [ ]:
from collections import Counter

In [ ]:
vocab = dataset.get_vocab()
idx_to_word = {v:k for k, v in vocab.items()}
len(vocab)

In [ ]:
token_appearances = Counter()
for report in dataset.iter_reports_only():
    for token in report['tokens_idxs']:
        token_appearances[token] += 1
len(token_appearances)

### K words with most appearances

In [ ]:
top_k_words = lambda k: tuple(zip(*sorted(
    token_appearances.items(),
    key=lambda x:x[1],
    reverse=True,
)[:k]))[0]

In [ ]:
word_tokens = top_k_words(800)
len(word_tokens)

In [ ]:
words = [idx_to_word[token] for token in word_tokens]
len(words), words[:10]

In [ ]:
EMBEDDING_NAME = str(len(words))

### All words

In [ ]:
words = [w for w in idx_to_word.values() if w not in ('PAD', 'START', 'END', 'UNK')]
len(words), words[:10]

In [ ]:
word_tokens = [vocab[word] for word in words]
len(word_tokens)

In [ ]:
EMBEDDING_NAME = 'all'

## Get organs and diseases

In [ ]:
%run ../datasets/common/sentences2organs/compute.py
%run ../datasets/common/constants.py

In [ ]:
organs_onehot, warnings = find_organs_for_sentences(words)
neutral_words = set(warnings['all-empty'])
len(organs_onehot), len(neutral_words)

In [ ]:
header = ['word', 'organ', *JSRT_ORGANS]
metadata = [
    (word, get_main_organ(one_hot, word, warnings), *one_hot)
    for word, one_hot in zip(words, organs_onehot)
]
len(header), len(metadata)

### Filter only non-neutral organs

In [ ]:
metadata = [
    t
    for t in metadata
    if t[1] != 'neutral'
]
len(metadata)

In [ ]:
words = tuple(zip(*metadata))[0]
len(words)

In [ ]:
word_tokens = [vocab[tup[0]] for tup in metadata]
len(word_tokens)

In [ ]:
EMBEDDING_NAME = 'non-neutral'

## Write from model to TB

In [ ]:
%run ../tensorboard/__init__.py

In [ ]:
word_tokens = torch.tensor(word_tokens, device='cuda')
word_tokens.size()

In [ ]:
def calculate_embeddings(compiled_model):
    with torch.no_grad():
        embeddings = compiled_model.model.decoder.word_embeddings(word_tokens)
    
    embeddings = embeddings.cpu().numpy()
    assert embeddings.shape == (len(word_tokens), 100)
    
    return embeddings

In [ ]:
def write_embeddings_from_compiled_model(run_id, compiled_model, embeddings):
    assert compiled_model.metadata['run_id'] == run_id.to_dict()

    writer = SummaryWriter(get_tb_large_log_folder(run_id))

    writer.add_embedding(
        embeddings,
        metadata=metadata,
        metadata_header=header,
        global_step=compiled_model.get_current_epoch(),
        tag=f'word_embeddings_{len(embeddings)}',
    )

    writer.close()

### Calculate for four base models

In [ ]:
run_id, compiled_model = _load_model_wrapper('0426_143345')
emb_h = calculate_embeddings(compiled_model)
# write_embeddings_from_compiled_model(run_id, compiled_model, emb_h)
emb_h.shape

In [ ]:
run_id, compiled_model = _load_model_wrapper('0426_221511')
emb_h_att = calculate_embeddings(compiled_model)
# write_embeddings_from_compiled_model(run_id, compiled_model, emb_h_att)
emb_h_att.shape

In [ ]:
run_id, compiled_model = _load_model_wrapper('0417_132754') # 0501_201357
emb_flat = calculate_embeddings(compiled_model)
# write_embeddings_from_compiled_model(run_id, compiled_model, emb_flat)
emb_flat.shape

In [ ]:
run_id, compiled_model = _load_model_wrapper('0418_102603')  # 0501_212955
emb_flat_att = calculate_embeddings(compiled_model)
# write_embeddings_from_compiled_model(run_id, compiled_model, emb_flat_att)
emb_flat_att.shape

### Plot distributions

In [ ]:
n_rows = 2
n_cols = 2

plt.figure(figsize=(n_cols * 7, n_rows * 5))

plt.subplot(n_rows, n_cols, 1)
plot_emb_distribution(emb_h, 'h-lstm')

plt.subplot(n_rows, n_cols, 2)
plot_emb_distribution(emb_h_att, 'h-lstm-att')

plt.subplot(n_rows, n_cols, 3)
plot_emb_distribution(emb_flat, 'lstm')

plt.subplot(n_rows, n_cols, 4)
plot_emb_distribution(emb_flat_att, 'lstm-att')

## Write from random layer

In [ ]:
emb_layer = nn.Embedding(len(vocab), 100, 0).to('cuda')
with torch.no_grad():
    embeddings = emb_layer(word_tokens)
    
embeddings.size()

In [ ]:
embeddings = embeddings.cpu().numpy()
embeddings.shape

In [ ]:
run_id = RunId('random_word_embedding', False, 'rg')
writer = SummaryWriter(get_tb_large_log_folder(run_id))

In [ ]:
writer.add_embedding(
    embeddings,
    metadata=metadata,
    metadata_header=header,
    global_step=0,
    tag=f'word_embeddings_{len(words)}',
)

In [ ]:
writer.close()

## Glove

In [ ]:
import torchtext

In [ ]:
glove = torchtext.vocab.GloVe(name='6B', dim=100)

In [ ]:
missing_words = [word for word in words if word not in glove.stoi]
len(missing_words)

In [ ]:
embeddings = glove.get_vecs_by_tokens(words)
embeddings.size()

In [ ]:
embeddings = embeddings.cpu().numpy()
embeddings.shape

In [ ]:
run_id = RunId('glove', False, 'rg')
writer = SummaryWriter(get_tb_large_log_folder(run_id))

writer.add_embedding(
    embeddings,
    metadata=metadata,
    metadata_header=header,
    global_step=0,
    tag=f'word_embeddings_{len(words)}',
)

writer.close()

In [ ]:
flat_embeddings = embeddings.flatten()
flat_embeddings.shape

In [ ]:
plot_emb_distribution(flat_embeddings)

## Load rad-glove

In [ ]:
%run ../models/report_generation/word_embedding.py

In [ ]:
radglove = RadGlove()
len(radglove)

In [ ]:
radglove['number']

### Write rad-glove to TB

In [ ]:
missing_words = [word for word in words if word not in radglove]
len(missing_words)

In [ ]:
embeddings = torch.stack([
    radglove[token] if token in radglove else torch.zeros(100)
    for token in words
], dim=0)
embeddings.size()

In [ ]:
embeddings = embeddings.cpu().numpy()
embeddings.shape

In [ ]:
run_id = RunId('radglove', False, 'rg')
writer = SummaryWriter(get_tb_large_log_folder(run_id))

writer.add_embedding(
    embeddings,
    metadata=metadata,
    metadata_header=header,
    global_step=0,
    tag=f'word_embeddings_{EMBEDDING_NAME}',
)

writer.close()

In [ ]:
plot_emb_distribution(embeddings, 'radglove')